# Natural Language Processing (NLP)
Språkteknologi (NLP på engelsk) fikk en boost etter at [Transformer-arkitekturen](https://arxiv.org/abs/1706.03762) ble introdusert i 2017. Denne arkitekturen danner grunnlaget for de aller fleste tjenestene som kombinerer AI og språk idag, der ChatGPT er et godt eksempel. 

Siden da har [Hugging Face](huggingface.co) dukket opp som plattform som forsøker å tilgjengeliggjøre datasett, implementasjoner og erfaringer innen NLP. Av den grunn vil denne notebooken ta i bruk bibliotekene som tilbys fra dem. 

## Fyll inn det manglende ordet

In [1]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='NbAiLab/nb-bert-base')

In [4]:
res = unmasker("Sykehusene bruker elektronisk [MASK].")
res

[{'score': 0.5338510274887085,
  'token': 19061,
  'token_str': 'journal',
  'sequence': 'Sykehusene bruker elektronisk journal.'},
 {'score': 0.10540317744016647,
  'token': 15623,
  'token_str': 'katalog',
  'sequence': 'Sykehusene bruker elektronisk katalog.'},
 {'score': 0.08050142973661423,
  'token': 11841,
  'token_str': 'post',
  'sequence': 'Sykehusene bruker elektronisk post.'},
 {'score': 0.023109378293156624,
  'token': 54332,
  'token_str': 'kode',
  'sequence': 'Sykehusene bruker elektronisk kode.'},
 {'score': 0.02191219851374626,
  'token': 27246,
  'token_str': 'multimedia',
  'sequence': 'Sykehusene bruker elektronisk multimedia.'}]

## Sammenlikne setninger

In [5]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('NbAiLab/nb-sbert-base')

In [8]:
anchor = "Jeg liker å spise is."
anchor_embedding = model.encode(anchor)
print(anchor_embedding.shape)

(768,)


In [9]:
sentences = ["Is er veldig godt", "Norske sykehus bruker elektronisk pasientjournal."]
sentence_embeddings = model.encode(sentences)
print(sentence_embeddings.shape)

(2, 768)


In [10]:
similarities_with_anchor = []
for emb in sentence_embeddings:
    cos_sim = util.cos_sim(anchor_embedding, emb).item()
    similarities_with_anchor.append(cos_sim)

print(f"Similarities with the sentence '{anchor}':")
for i, sentence in enumerate(sentences):
    print(f"{sentence}: {similarities_with_anchor[i]:.3f}")

Similarities with the sentence 'Jeg liker å spise is.':
Is er veldig godt: 0.733
Norske sykehus bruker elektronisk pasientjournal.: 0.061


## Likhetssøk i database

In [11]:
from autofaiss import build_index

In [12]:
sentences = [ 
    "På norske sykehus brukes det elektroniske pasientjournaler", 
    "Bier er viktige for pollinering av mange av våre matvekster.",  
    "Mange nordmenn elsker å gå på ski om vinteren.",  
    "Astronomi er studiet av universet og dets himmellegemer.",  
    "Sjokolade er en populær godbit som lages fra kakaobønner.",  
    "Fotball er en av de mest populære sportene i verden.",  
    "Paris er kjent for sin arkitektur, mat og mote.",  
    "Elefanter er det største landdyret på jorden.",  
    "Bøker er en viktig kilde til kunnskap og underholdning.",  
    "Musikk er en universell form for kunst som uttrykker følelser og ideer.",  
    "Klimaendringer er en stor utfordring for verden i dag.",
    "Taco er vanlig å spise på fredager"
]  

embeddings = model.encode(sentences)

In [13]:
index, index_infos = build_index(embeddings, save_on_disk=False)

2023-10-27 14:40:24,899 [INFO]: Using 20 omp threads (processes), consider increasing --nb_cores if you have more
2023-10-27 14:40:24,902 [INFO]: Launching the whole pipeline 10/27/2023, 14:40:24
2023-10-27 14:40:24,903 [INFO]: Reading total number of vectors and dimension 10/27/2023, 14:40:24
100%|██████████| 1/1 [00:00<00:00, 3637.73it/s]
2023-10-27 14:40:24,944 [INFO]: There are 12 embeddings of dim 768
2023-10-27 14:40:24,945 [INFO]: >>> Finished "Reading total number of vectors and dimension" in 0.0412 secs
2023-10-27 14:40:24,946 [INFO]: 	Compute estimated construction time of the index 10/27/2023, 14:40:24
2023-10-27 14:40:24,947 [INFO]: 		-> Train: 16.7 minutes
2023-10-27 14:40:24,948 [INFO]: 		-> Add: 0.0 seconds
2023-10-27 14:40:24,949 [INFO]: 		Total: 16.7 minutes
2023-10-27 14:40:24,950 [INFO]: 	>>> Finished "Compute estimated construction time of the index" in 0.0031 secs
2023-10-27 14:40:24,951 [INFO]: 	Checking that your have enough memory available to create the index 1